<a href="https://colab.research.google.com/github/PierreM31/Kaggle_Challenge_NYC/blob/master/model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import lib

### Import TensorFlow GPU

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


### Other import


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import floor, ceil
import seaborn as sns
import random
from keras.callbacks import LambdaCallback
% matplotlib inline
plt.style.use('seaborn-whitegrid')

Using TensorFlow backend.


## Import data from BigQuerry 

In [0]:
#TODO
"""from google.cloud import bigquery

client = bigquery.Client(project='bigquery-public-data')"""

## Import data directly from Colab


In [5]:
df_train = pd.read_csv('train.csv', nrows=500_000, parse_dates=["pickup_datetime"])
df_train.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2012-01-18 10:15:28.0000001,5.3,2012-01-18 10:15:28,-73.999906,40.727304,-74.001324,40.736911,1
1,2012-09-06 14:48:00.00000030,21.5,2012-09-06 14:48:00,-73.974438,40.754583,-73.979488,40.764328,5
2,2009-05-02 22:19:14.0000002,10.5,2009-05-02 22:19:14,0.000000,0.000000,0.000000,0.000000,1
3,2009-10-31 12:10:59.0000001,5.7,2009-10-31 12:10:59,0.000000,0.000000,0.000000,0.000000,1
4,2012-10-30 12:33:41.0000001,4.0,2012-10-30 12:33:41,-73.989309,40.763072,-73.988742,40.758697,1


## Preprocessing

In [6]:
print('Sample size before preprocessing: %d' % len(df_train))

Sample size before preprocessing: 174752


In [0]:
# drop-out negative fare
df_train = df_train[df_train.fare_amount>=0]

# drop-out non full line
df_train = df_train.dropna(how = 'any', axis = 'rows')

In [0]:
# minimum and maximum longitude test set
min(df_train.pickup_longitude.min(), df_train.dropoff_longitude.min()), \
max(df_train.pickup_longitude.max(), df_train.dropoff_longitude.max())

# minimum and maximum latitude test set
min(df_train.pickup_latitude.min(), df_train.dropoff_latitude.min()), \
max(df_train.pickup_latitude.max(), df_train.dropoff_latitude.max())

def select_within_boundingbox(df, BB):
    return (df.pickup_longitude >= BB[0]) & (df.pickup_longitude <= BB[1]) & \
           (df.pickup_latitude >= BB[2]) & (df.pickup_latitude <= BB[3]) & \
           (df.dropoff_longitude >= BB[0]) & (df.dropoff_longitude <= BB[1]) & \
           (df.dropoff_latitude >= BB[2]) & (df.dropoff_latitude <= BB[3])
  
# load image of NYC map
BB = (-74.5, -72.8, 40.5, 41.8)
nyc_map = plt.imread('https://aiblog.nl/download/nyc_-74.5_-72.8_40.5_41.8.png')

# load extra image to zoom in on NYC
BB_zoom = (-74.3, -73.7, 40.5, 40.9)
nyc_map_zoom = plt.imread('https://aiblog.nl/download/nyc_-74.3_-73.7_40.5_40.9.png')

df_train = df_train[select_within_boundingbox(df_train, BB)]

In [9]:
# Eead nyc mask and turn into boolean map with
# land = True, water = False
nyc_mask = plt.imread('https://aiblog.nl/download/nyc_mask-74.5_-72.8_40.5_41.8.png')[:,:,0] > 0.9

# Translate longitude/latitude coordinate into image xy coordinate
def lonlat_to_xy(longitude, latitude, dx, dy, BB):
    return (dx*(longitude - BB[0])/(BB[1]-BB[0])).astype('int'), \
           (dy - dy*(latitude - BB[2])/(BB[3]-BB[2])).astype('int')
  
pickup_x, pickup_y = lonlat_to_xy(df_train.pickup_longitude, df_train.pickup_latitude, 
                                  nyc_mask.shape[1], nyc_mask.shape[0], BB)
dropoff_x, dropoff_y = lonlat_to_xy(df_train.dropoff_longitude, df_train.dropoff_latitude, 
                                  nyc_mask.shape[1], nyc_mask.shape[0], BB)


idx = (nyc_mask[pickup_y, pickup_x] & nyc_mask[dropoff_y, dropoff_x])
print("Number of trips in water: {}".format(np.sum(~idx)))


def remove_datapoints_from_water(df):
    def lonlat_to_xy(longitude, latitude, dx, dy, BB):
        return (dx*(longitude - BB[0])/(BB[1]-BB[0])).astype('int'), \
               (dy - dy*(latitude - BB[2])/(BB[3]-BB[2])).astype('int')
      
    # define bounding box
    BB = (-74.5, -72.8, 40.5, 41.8)
    
    # read nyc mask and turn into boolean map with
    # land = True, water = False
    nyc_mask = plt.imread('https://aiblog.nl/download/nyc_mask-74.5_-72.8_40.5_41.8.png')[:,:,0] > 0.9
    
    # calculate for each lon,lat coordinate the xy coordinate in the mask map
    pickup_x, pickup_y = lonlat_to_xy(df.pickup_longitude, df.pickup_latitude, 
                                      nyc_mask.shape[1], nyc_mask.shape[0], BB)
    dropoff_x, dropoff_y = lonlat_to_xy(df.dropoff_longitude, df.dropoff_latitude, 
                                      nyc_mask.shape[1], nyc_mask.shape[0], BB)    
    # calculate boolean index
    idx = nyc_mask[pickup_y, pickup_x] & nyc_mask[dropoff_y, dropoff_x]
    
    # return only datapoints on land
    return df[idx]
  
  

df_train = remove_datapoints_from_water(df_train)

Number of trips in water: 32


In [10]:
print('Sample size after preprocessing: %d' % len(df_train))

Sample size after preprocessing: 171088


## Feature engineering

### Manhattan distance

In [11]:
# Given a dataframe, add two new features 'abs_diff_longitude' and 'abs_diff_latitude' reprensenting the vector from
# the pickup location to the dropoff location

def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(df_train)


def convert_kilometers(train_df):
    train_df['abs_diff_longitude'] = train_df.abs_diff_longitude * 79.1
    train_df['abs_diff_latitude'] = train_df.abs_diff_latitude * 111
    
convert_kilometers(df_train)


### Angle difference between north, and manhattan roadways
meas_ang = 0.506 # 29 degrees = 0.506 radians
import math

## adding extra features
def add_extra_manh_features(df):
    df['euclidean_distance'] = (df.abs_diff_latitude**2 + df.abs_diff_longitude**2)**0.5  
    df['delta_manh_long'] = (df.euclidean_distance*np.sin(np.arctan(df.abs_diff_longitude / df.abs_diff_latitude)-meas_ang)).abs()
    df['delta_manh_lat'] = (df.euclidean_distance*np.cos(np.arctan(df.abs_diff_longitude / df.abs_diff_latitude)-meas_ang)).abs()
    df['manh_distance'] = df.delta_manh_long + df.delta_manh_lat
    df['euclidean_error'] = (df.manh_distance - df.euclidean_distance)*100 /  df.euclidean_distance

    
add_extra_manh_features(df_train)

tmp = len(df_train)
df_train = df_train.dropna(how = 'any', axis = 'rows')
print('Raws where distante = NaN: %d' % (tmp-len(df_train)))

df_train.head()

Raws where distante = NaN: 1843


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,abs_diff_longitude,abs_diff_latitude,euclidean_distance,delta_manh_long,delta_manh_lat,manh_distance,euclidean_error
0,2012-01-18 10:15:28.0000001,5.3,2012-01-18 10:15:28,-73.999906,40.727304,-74.001324,40.736911,1,0.112164,1.066377,1.072260,0.418746,0.987113,1.405859,31.111806
1,2012-09-06 14:48:00.00000030,21.5,2012-09-06 14:48:00,-73.974438,40.754583,-73.979488,40.764328,5,0.399455,1.081695,1.153095,0.174879,1.139757,1.314636,14.009324
4,2012-10-30 12:33:41.0000001,4.0,2012-10-30 12:33:41,-73.989309,40.763072,-73.988742,40.758697,1,0.044850,0.485625,0.487692,0.196144,0.446509,0.642654,31.774575
5,2014-07-21 22:15:00.000000159,30.0,2014-07-21 22:15:00,-73.875200,40.773922,-73.971135,40.753697,1,7.588458,2.244975,7.913572,5.549451,5.641650,11.191100,41.416557
6,2011-02-25 23:35:00.000000154,13.3,2011-02-25 23:35:00,-74.008035,40.705807,-73.987173,40.729433,5,1.650184,2.622486,3.098474,0.172327,3.093678,3.266005,5.406900


## Build the network

In [15]:
print(df_train.pickup_datetime[5].dayofweek)

0


In [0]:
def getYear(x):
    return x.year%2000
  
def getMonth(x):
  return x.month

def getHour(x):
  return x.hour  

def getHourCos(x):
  return math.cos(2* math.pi * x.hour/24)  

def getHourSin(x):
  return math.sin(2* math.pi * x.hour/24)  

def getDayOfWeek(x,day):
  #tmp = tf.keras.backend.one_hot(x.dayofweek, 7)
  if x.dayofweek == day:
    return 1
  else :
    return 0


train_x = pd.DataFrame()
train_x['pickup_longitude'] = df_train.pickup_longitude
train_x['pickup_latitude'] = df_train.pickup_latitude
train_x['dropoff_longitude'] = df_train.dropoff_longitude
train_x['dropoff_latitude'] = df_train.dropoff_latitude
train_x['passenger_count'] = df_train.passenger_count
train_x['manh_distance'] = df_train.manh_distance
train_x['pickup_year'] = df_train.pickup_datetime.apply(getYear)
train_x['pickup_month'] = df_train.pickup_datetime.apply(getMonth)
train_x['pickup_hour'] = df_train.pickup_datetime.apply(getHour)

#train_x['pickup_hour_cos'] = df_train.pickup_datetime.apply(getHourCos)
#train_x['pickup_hour_sin'] = df_train.pickup_datetime.apply(getHourSin)

"""train_x['pickup_day_of_week_mon'] = df_train.pickup_datetime.apply(lambda x: getDayOfWeek(x,0))
train_x['pickup_day_of_week_tue'] = df_train.pickup_datetime.apply(lambda x: getDayOfWeek(x,1))
train_x['pickup_day_of_week_wed'] = df_train.pickup_datetime.apply(lambda x: getDayOfWeek(x,2))
train_x['pickup_day_of_week_thu'] = df_train.pickup_datetime.apply(lambda x: getDayOfWeek(x,3))
train_x['pickup_day_of_week_fri'] = df_train.pickup_datetime.apply(lambda x: getDayOfWeek(x,4))
train_x['pickup_day_of_week_sat'] = df_train.pickup_datetime.apply(lambda x: getDayOfWeek(x,5))
train_x['pickup_day_of_week_sun'] = df_train.pickup_datetime.apply(lambda x: getDayOfWeek(x,6))"""


train_y = df_train.fare_amount

train_size = 0.999 #0.00009 #0.99996
train_cnt = floor(train_x.shape[0] * train_size)
rand = random.randint(0,train_x.shape[0]-train_cnt)
x_train = train_x.iloc[0:train_cnt].values
y_train = train_y.iloc[0:train_cnt].values
x_test = train_x.iloc[train_cnt:].values
y_test = train_y.iloc[train_cnt:].values

In [65]:
inputs = tf.keras.Input(shape=(9,), name='input_layer')

x = tf.keras.layers.Dense(1024, activation='relu')(inputs)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
predictions = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])
print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[5].get_weights()))
#model.fit(x_train, y_train, batch_size=32, epochs=20,callbacks = [print_weights])
model.fit(x_train, y_train, batch_size=32, epochs=6)

Epoch 1/6
169075/169075 [==============================] - 31s 183us/step - loss: 25.1797 - acc: 0.0000e+00
Epoch 2/6
169075/169075 [==============================] - 30s 177us/step - loss: 20.0558 - acc: 0.0000e+00
Epoch 3/6
169075/169075 [==============================] - 30s 179us/step - loss: 19.6555 - acc: 0.0000e+00
Epoch 4/6
169075/169075 [==============================] - 30s 176us/step - loss: 19.3473 - acc: 0.0000e+00
Epoch 5/6
169075/169075 [==============================] - 30s 178us/step - loss: 19.2409 - acc: 0.0000e+00
Epoch 6/6
169075/169075 [==============================] - 30s 175us/step - loss: 19.1888 - acc: 0.0000e+00


In [67]:
model.evaluate(x_train, y_train)


169075/169075 [==============================] - 11s 63us/step


[19.267743133822435, 0.0]

In [89]:

print (y_test[0:15])
prediction = model.predict(x_test[0:15])
print (prediction)
erreurTotale = 0
for i in range(15):
  erreurTotale += abs(y_test[i] - prediction[i])[0]
  
print('Erreur moyenne : ' + str(erreurTotale/15))


[36.83  5.    9.3  39.    8.1   6.9   4.5   5.7  16.    8.    8.   10.5
  8.1   5.7   7.3 ]
[[31.099182 ]
 [ 6.2038965]
 [10.259786 ]
 [34.026577 ]
 [ 7.3879194]
 [ 6.5695753]
 [ 5.1695356]
 [ 6.1133957]
 [13.714487 ]
 [ 6.252534 ]
 [ 7.308263 ]
 [11.900133 ]
 [ 8.830034 ]
 [ 5.7901754]
 [ 7.8193026]]
Erreur moyenne : 1.4971815427144368
